# BIBLIOTECAS

Instalações

In [1]:
!pip install ahpy

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.9/45.9 kB 2.0 MB/s eta 0:00:00


In [2]:
!pip install xlsxwriter  # Install the missing module

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 165.1/165.1 kB 2.8 MB/s eta 0:00:00


Importação das bibliotecas

In [3]:
from ahpy import Compare
import pandas as pd
from scipy.stats import zscore
from sklearn.preprocessing import MinMaxScaler, RobustScaler
from google.colab import drive
from google.colab import files
import xlsxwriter # Import the module

# Cálculo do AHP

**Objetivo:**
Destacar regiões com maior vulnerabilidade socioambiental

**A escala fundamental para comparações em pares:**

|Intensidade de importância | Definição | Explicação|
| --- | --- | --- |
| 1 | Igual importância | Os dois elementos contribuem igualmente para o objetivo.|
| 2 | Importância moderada | A experiência e o impacto favorecem ligeiramente um elemento em detrimento do outro.|
| 3 | Alta importância | A experiência e o impacto favorecem fortemente um elemento em detrimento de outro.|
| 4 | Muita Alta importância | Um elemento é fortemente favorecido em relação a outro e seu domínio é demonstrado na prática.|
| 5 | Extremamente importante | A evidência que favorece um elemento em detrimento de outro é da mais alta ordem possível de afirmação.|

## Comparação dos Critérios


Nomeclatura:
*   C - Criticidade
*   S - Suportabilidade
*   H - Histórico incidentes



In [156]:
top_level_comparisons = {
    ('C', 'S'): 3.0,
    ('C', 'H'): 5.0,
    ('S', 'H'): 4.0
}

In [157]:
# No Compare do nível superior, passamos os submodelos em "add_children":
top_level = Compare(
    name='Criterios',
    comparisons=top_level_comparisons,
    precision=3,
    random_index='saaty'
)

In [158]:
print(top_level.target_weights)

{'C': 0.627, 'S': 0.28, 'H': 0.094}


In [159]:
print(top_level.consistency_ratio)

0.082


## Comparação dos subcritérios (indicadores)

Função para calcular o peso global

In [8]:
# Calcula pesos globais (peso global = peso do subcritério local * peso do critério principal)
def calcular_pesos_globais(subcriterios, peso_criterio):
    return {ind: round(peso * peso_criterio, 4) for ind, peso in subcriterios.items()}

### Subcritério da Criticidade

Os subcritérios da Criticidade:
1.   **[ Ind1 ]** = Mulheres chefes de família sem cônjuge e com pelo menos um filho.
2.   **[ Ind2 ]** = Média de moradores por domicílio.
3.   **[ Ind3 ]** = Crianças de 0 a 14 anos.
4.   **[ Ind4 ]** = Idosos com mais de 65 anos.
5.   **[ Ind5 ]** = Pessoas residentes por cor ou raça - pessoas não brancas.
6.   **[ Ind6 ]** = Famílias permanentes com três ou mais residentes.
7.   **[ Ind7 ]** = Famílias residentes em favelas e comunidades urbanas.
8.   **[ Ind8 ]** = Taxa de analfabetos das pessoas de 15 anos ou mais.

In [160]:
crit_comparisons = {
    ('Ind1', 'Ind2'): 5.0,
    ('Ind1', 'Ind3'): 1/2.0,
    ('Ind1', 'Ind4'): 1/2.0,
    ('Ind1', 'Ind5'): 2.0,
    ('Ind1', 'Ind6'): 3.0,
    ('Ind1', 'Ind7'): 1/4.0,
    ('Ind1', 'Ind8'): 3.0,

    ('Ind2', 'Ind3'): 1/3.0,
    ('Ind2', 'Ind4'): 1/3.0,
    ('Ind2', 'Ind5'): 1/2.0,
    ('Ind2', 'Ind6'): 1.0,
    ('Ind2', 'Ind7'): 1/5.0,
    ('Ind2', 'Ind8'): 3.0,

    ('Ind3', 'Ind4'): 1.0,
    ('Ind3', 'Ind5'): 3.0,
    ('Ind3', 'Ind6'): 2.0,
    ('Ind3', 'Ind7'): 1/3.0,
    ('Ind3', 'Ind8'): 4.0,

    ('Ind4', 'Ind5'): 3.0,
    ('Ind4', 'Ind6'): 2.0,
    ('Ind4', 'Ind7'): 1/3.0,
    ('Ind4', 'Ind8'): 4.0,

    ('Ind5', 'Ind6'): 4.0,
    ('Ind5', 'Ind7'): 1.0,
    ('Ind5', 'Ind8'): 2.0,

    ('Ind6', 'Ind7'): 1/5.0,
    ('Ind6', 'Ind8'): 2.0,

    ('Ind7', 'Ind8'): 5.0
}

In [161]:
criticidade = Compare(
    name='Criticidade',
    comparisons=crit_comparisons,
    precision=3,
    random_index='saaty'
)

In [162]:
print(criticidade.target_weights)

{'Ind7': 0.296, 'Ind3': 0.162, 'Ind4': 0.162, 'Ind1': 0.127, 'Ind5': 0.111, 'Ind6': 0.054, 'Ind2': 0.053, 'Ind8': 0.036}


In [163]:
print(criticidade.consistency_ratio)

0.079


Prioridade global

In [164]:
pesos_criterios = top_level.target_weights
sub_criticidade = criticidade.target_weights

In [165]:
globais_criticidade = calcular_pesos_globais(sub_criticidade, pesos_criterios['C'])

In [166]:
print(globais_criticidade)

{'Ind7': 0.1856, 'Ind3': 0.1016, 'Ind4': 0.1016, 'Ind1': 0.0796, 'Ind5': 0.0696, 'Ind6': 0.0339, 'Ind2': 0.0332, 'Ind8': 0.0226}


### Subcritério da Suportabilidade

Dos subcritérios do conjunto Suportabilidade:
1.   **[ Ind9 ]** = Domicílios permanentes com destino de lixo irregular.
2.   **[ Ind10 ]** = Domicílios permanentes com esgotamento irregular.
3.   **[ Ind11 ]** = Domicílios permanentes que não possuem ligação de água na rede geral.
4.   **[ Ind12 ]** = Razão população vs estabelecimentos de saúde.
5.   **[ Ind13 ]** = Domicílios sem acesso a internet.


In [167]:
sup_comparisons = {
    ('Ind9', 'Ind10'): 1.0,
    ('Ind9', 'Ind11'): 3.0,
    ('Ind9', 'Ind12'): 2.0,
    ('Ind9', 'Ind13'): 5.0,

    ('Ind10', 'Ind11'): 4.0,
    ('Ind10', 'Ind12'): 2.0,
    ('Ind10', 'Ind13'): 5.0,

    ('Ind11', 'Ind12'): 1/3.0,
    ('Ind11', 'Ind13'): 3.0,

    ('Ind12', 'Ind13'): 4.0
}


In [168]:
suporte = Compare(
    name='Suporte',
    comparisons=sup_comparisons,
    precision=3,
    random_index='saaty'
)

In [169]:
print(suporte.target_weights)

{'Ind10': 0.333, 'Ind9': 0.314, 'Ind12': 0.201, 'Ind11': 0.1, 'Ind13': 0.052}


In [170]:
print(suporte.consistency_ratio)

0.027


Prioridade global

In [171]:
pesos_criterios = top_level.target_weights
sub_suporte = suporte.target_weights

In [172]:
globais_suportabilidade = calcular_pesos_globais(sub_suporte, pesos_criterios['S'])

In [173]:
print(globais_suportabilidade)

{'Ind10': 0.0932, 'Ind9': 0.0879, 'Ind12': 0.0563, 'Ind11': 0.028, 'Ind13': 0.0146}


### Subcritérios do Histórico de Incidentes

Dos indicadores do conjunto Historico:
1.   **[ Ind14 ]** = Quantidade de desastre hidrológicos registrados.
2.   **[ Ind15 ]** = População afetada pelos desastres.
3.   **[ Ind16 ]** = Unidades habitacionais afetadas pelos desastres.
4.   **[ Ind17 ]** = Instalações públicas de saúde afetadas pelos desastres.


In [174]:
hist_comparisons = {
    ('Ind14', 'Ind15'): 1/3.0,
    ('Ind14', 'Ind16'): 1/4.0,
    ('Ind14', 'Ind17'): 2.0,

    ('Ind15', 'Ind16'): 1.0,
    ('Ind15', 'Ind17'): 3.0,

    ('Ind16', 'Ind17'): 5.0
}

In [175]:
historico = Compare(
    name='Historico',
    comparisons=hist_comparisons,
    precision=3,
    random_index='saaty'
)

In [176]:
print(historico.target_weights)

{'Ind16': 0.429, 'Ind15': 0.353, 'Ind14': 0.131, 'Ind17': 0.088}


In [177]:
print(historico.consistency_ratio)

0.025


Prioridade global

In [178]:
pesos_criterios = top_level.target_weights
sub_historico = historico.target_weights

In [179]:
globais_historico = calcular_pesos_globais(sub_historico, pesos_criterios['H'])

In [180]:
print(globais_historico)

{'Ind16': 0.0403, 'Ind15': 0.0332, 'Ind14': 0.0123, 'Ind17': 0.0083}


### Validação dos resultados

In [181]:
# Conferência da soma total (deve ser ~1)
soma_total = (
    sum(globais_criticidade.values())
    + sum(globais_suportabilidade.values())
    + sum(globais_historico.values())
)
print("Soma dos pesos globais:", round(soma_total, 6))

Soma dos pesos globais: 1.0018


# Aplicação dos pesos AHP aos indicadores - Com outliers

## Auxiliares

Acessar o drive

In [214]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [215]:
minmax_scaler = MinMaxScaler(feature_range=(0, 1))
robust_scaler = RobustScaler()

In [216]:
df_global = {}

funções auxiliares

In [217]:
def normalizar_dataframe(xlsx):
    df_global = {}

    # Itera sobre todas as abas do arquivo Excel
    for sheet_name in xlsx.sheet_names:
      # Lê cada aba e armazena no dicionário
      df = xlsx.parse(sheet_name)

      # Verifica se a coluna "CIDADE" existe no DataFrame
      if 'CIDADE' in df.columns:
          # Remove o " (RJ)" da coluna "CIDADE"
          df['CIDADE'] = df['CIDADE'].str.replace(' (RJ)', '', regex=False)

      if 'VALOR' in df.columns:
        # 1) Aplicar RobustScaler que usa a mediana e o intervalo interquartil (IQR), sendo resistente a outliers sem distorcer os dados.
        # link: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html
        df["VALOR_ROBUST"] = robust_scaler.fit_transform(df[["VALOR"]])

        # 2) Normalização Min-Max (após escalonamento robusto)
        # link: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html
        df['VALOR_NORMAL'] = minmax_scaler.fit_transform(df[['VALOR_ROBUST']])

      # Armazena o DataFrame no dicionário
      df_global[sheet_name] = df

    return df_global

In [218]:
def sum_criterio(dfs_dic):
  # Passo 1: Concatenar todos os DataFrames em um único DataFrame
  df_concatenado = pd.concat(dfs_dic.values(), ignore_index=True)
  # Passo 2: Agrupar por 'CIDADE' e somar 'VALOR_FINAL'
  df_final = df_concatenado.groupby('CIDADE', as_index=False)['VALOR_FINAL'].sum()
  return df_final

## Acesso ao indicadores de criticidade

In [219]:
xlxs_criticidade = pd.ExcelFile('/content/drive/MyDrive/BCC/BRUNA_LIMA/DATASETS/IVSA/IND_CRITICIDADE.xlsx')

In [220]:
dfs_criticidade = normalizar_dataframe(xlxs_criticidade)

In [221]:
df_test = dfs_criticidade['Ind1']
df_test.head(10)

,CIDADE,VALOR,VALOR_ROBUST,VALOR_NORMAL
0,Angra dos Reis,8297,1.085693,0.021375
1,Aperibé,541,-0.213687,0.000716
2,Araruama,6583,0.798542,0.016809
3,Areal,606,-0.202798,0.000890
4,Armação dos Búzios,1621,-0.032753,0.003593
5,Arraial do Cabo,1517,-0.050176,0.003316
6,Barra do Piraí,5815,0.669878,0.014764
7,Barra Mansa,9827,1.342017,0.025450
8,Belford Roxo,27469,4.297621,0.072439
9,Bom Jardim,1312,-0.084520,0.002770


Aplicar os pesos do AHP

In [222]:
# Itera sobre os DataFrames e aplica o peso correspondente
for sheet_name, df in dfs_criticidade.items():
  if sheet_name in criticidade.target_weights:  # Verifica se a aba tem um peso definido
    peso = criticidade.target_weights[sheet_name] # Pega o peso correspondente
    df['VALOR_FINAL'] = df['VALOR_NORMAL'] * peso  # Aplica o peso

In [223]:
df_test = dfs_criticidade['Ind1']
df_test.head(10)

,CIDADE,VALOR,VALOR_ROBUST,VALOR_NORMAL,VALOR_FINAL
0,Angra dos Reis,8297,1.085693,0.021375,0.002715
1,Aperibé,541,-0.213687,0.000716,0.000091
2,Araruama,6583,0.798542,0.016809,0.002135
3,Areal,606,-0.202798,0.000890,0.000113
4,Armação dos Búzios,1621,-0.032753,0.003593,0.000456
5,Arraial do Cabo,1517,-0.050176,0.003316,0.000421
6,Barra do Piraí,5815,0.669878,0.014764,0.001875
7,Barra Mansa,9827,1.342017,0.025450,0.003232
8,Belford Roxo,27469,4.297621,0.072439,0.009200
9,Bom Jardim,1312,-0.084520,0.002770,0.000352


Valor total do critério criticidade

In [224]:
df_global_criticidade = sum_criterio(dfs_criticidade)

In [225]:
df_global_criticidade.head(10)

,CIDADE,VALOR_FINAL
0,Angra dos Reis,0.066839
1,Aperibé,0.023525
2,Araruama,0.053583
3,Areal,0.052738
4,Armação dos Búzios,0.031745
5,Arraial do Cabo,0.035618
6,Barra Mansa,0.050505
7,Barra do Piraí,0.038431
8,Belford Roxo,0.089259
9,Bom Jardim,0.035152


## Acesso ao indicadores de suportabilidade

In [226]:
xlxs_suportabilidade = pd.ExcelFile('/content/drive/MyDrive/BCC/BRUNA_LIMA/DATASETS/IVSA/IND_SUPORTABILIDADE.xlsx')

In [227]:
dfs_suportabilidade = normalizar_dataframe(xlxs_suportabilidade)

In [228]:
df_test = dfs_suportabilidade['Ind9']
df_test.head(10)

,CIDADE,VALOR,VALOR_ROBUST,VALOR_NORMAL
0,Angra dos Reis,406,0.096705,0.019418
1,Aperibé,168,-0.310655,0.007658
2,Araruama,1522,2.006846,0.074559
3,Areal,59,-0.497219,0.002273
4,Armação dos Búzios,48,-0.516046,0.001729
5,Arraial do Cabo,46,-0.519469,0.001631
6,Barra do Piraí,631,0.481814,0.030535
7,Barra Mansa,171,-0.305520,0.007807
8,Belford Roxo,20252,34.065041,1.000000
9,Bom Jardim,173,-0.302097,0.007906


In [229]:
# Itera sobre os DataFrames e aplica o peso correspondente
for sheet_name, df in dfs_suportabilidade.items():
  if sheet_name in suporte.target_weights:  # Verifica se a aba tem um peso definido
    peso = suporte.target_weights[sheet_name] # Pega o peso correspondente
    df['VALOR_FINAL'] = df['VALOR_NORMAL'] * peso  # Aplica o peso

In [230]:
df_test = dfs_suportabilidade['Ind9']
df_test.head(10)

,CIDADE,VALOR,VALOR_ROBUST,VALOR_NORMAL,VALOR_FINAL
0,Angra dos Reis,406,0.096705,0.019418,0.006097
1,Aperibé,168,-0.310655,0.007658,0.002405
2,Araruama,1522,2.006846,0.074559,0.023412
3,Areal,59,-0.497219,0.002273,0.000714
4,Armação dos Búzios,48,-0.516046,0.001729,0.000543
5,Arraial do Cabo,46,-0.519469,0.001631,0.000512
6,Barra do Piraí,631,0.481814,0.030535,0.009588
7,Barra Mansa,171,-0.305520,0.007807,0.002451
8,Belford Roxo,20252,34.065041,1.000000,0.314000
9,Bom Jardim,173,-0.302097,0.007906,0.002482


Valor total do critério suportabilidade

In [231]:
df_global_suportabilidade = sum_criterio(dfs_suportabilidade)

In [232]:
df_global_suportabilidade.head(10)

,CIDADE,VALOR_FINAL
0,Angra dos Reis,0.070031
1,Aperibé,0.036685
2,Araruama,0.129721
3,Areal,0.021860
4,Armação dos Búzios,0.063616
5,Arraial do Cabo,0.075480
6,Barra Mansa,0.052309
7,Barra do Piraí,0.087768
8,Belford Roxo,0.581780
9,Bom Jardim,0.050191


## Acesso ao indicadores de Histórico de Incidente

In [233]:
xlxs_historico = pd.ExcelFile('/content/drive/MyDrive/BCC/BRUNA_LIMA/DATASETS/IVSA/IND_HISTORICO.xlsx')

In [234]:
dfs_historico = normalizar_dataframe(xlxs_historico)

In [235]:
df_test = dfs_historico['Ind14']
df_test.head(10)

,CIDADE,VALOR,VALOR_ROBUST,VALOR_NORMAL
0,Angra dos Reis,19,1.444444,0.375000
1,Aperibé,7,0.111111,0.125000
2,Araruama,2,-0.444444,0.020833
3,Areal,2,-0.444444,0.020833
4,Armação dos Búzios,1,-0.555556,0.000000
5,Arraial do Cabo,14,0.888889,0.270833
6,Barra do Piraí,13,0.777778,0.250000
7,Barra Mansa,7,0.111111,0.125000
8,Belford Roxo,4,-0.222222,0.062500
9,Bom Jesus do Itabapoana,20,1.555556,0.395833


In [236]:
# Itera sobre os DataFrames e aplica o peso correspondente
for sheet_name, df in dfs_historico.items():
  if sheet_name in historico.target_weights:  # Verifica se a aba tem um peso definido
    peso = historico.target_weights[sheet_name] # Pega o peso correspondente
    df['VALOR_FINAL'] = df['VALOR_NORMAL'] * peso  # Aplica o peso

In [237]:
df_test = dfs_historico['Ind14']
df_test.head(10)

,CIDADE,VALOR,VALOR_ROBUST,VALOR_NORMAL,VALOR_FINAL
0,Angra dos Reis,19,1.444444,0.375000,0.049125
1,Aperibé,7,0.111111,0.125000,0.016375
2,Araruama,2,-0.444444,0.020833,0.002729
3,Areal,2,-0.444444,0.020833,0.002729
4,Armação dos Búzios,1,-0.555556,0.000000,0.000000
5,Arraial do Cabo,14,0.888889,0.270833,0.035479
6,Barra do Piraí,13,0.777778,0.250000,0.032750
7,Barra Mansa,7,0.111111,0.125000,0.016375
8,Belford Roxo,4,-0.222222,0.062500,0.008188
9,Bom Jesus do Itabapoana,20,1.555556,0.395833,0.051854


Valor total do critério Histórico de Incidentes

In [238]:
df_global_historico = sum_criterio(dfs_historico)

In [239]:
df_global_historico.head(10)

,CIDADE,VALOR_FINAL
0,Angra dos Reis,0.170567
1,Aperibé,0.028677
2,Araruama,0.014302
3,Areal,0.004374
4,Armação dos Búzios,0.004659
5,Arraial do Cabo,0.038441
6,Barra Mansa,0.048901
7,Barra do Piraí,0.065743
8,Belford Roxo,0.241738
9,Bom Jesus do Itabapoana,0.141883


# Aplicação dos pesos AHP aos indicadores - Sem outliers

## Auxiliares

In [261]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [262]:
minmax_scaler = MinMaxScaler(feature_range=(0, 1))

In [263]:
df_global = {}

In [264]:
def normalizar_dataframe(xlsx):
    df_global = {}

    # Itera sobre todas as abas do arquivo Excel
    for sheet_name in xlsx.sheet_names:
      # Lê cada aba e armazena no dicionário
      df = xlsx.parse(sheet_name)

      # Verifica se a coluna "CIDADE" existe no DataFrame
      if 'CIDADE' in df.columns:
          # Remove o " (RJ)" da coluna "CIDADE"
          df['CIDADE'] = df['CIDADE'].str.replace(' (RJ)', '', regex=False)

      if 'VALOR' in df.columns:
        # Calcular Z-score
        df['Z-SCORE'] = zscore(df['VALOR'])

        # Definir outliers como valores com Z-score > 3 ou < -3
        outlier_threshold = 3
        df['OUTLIER'] = df['Z-SCORE'].abs() > outlier_threshold

        # Remover os municípios com outliers
        df_sem_outliers = df[~df["OUTLIER"]].copy()

        # 2) Normalização Min-Max (após escalonamento robusto)
        # link: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html
        df_sem_outliers['VALOR_NORMAL'] = minmax_scaler.fit_transform(df_sem_outliers[['VALOR']])

      # Armazena o DataFrame no dicionário
      df_global[sheet_name] = df_sem_outliers

    return df_global

In [265]:
def sum_criterio(dfs_dic):
  # Passo 1: Concatenar todos os DataFrames em um único DataFrame
  df_concatenado = pd.concat(dfs_dic.values(), ignore_index=True)
  # Passo 2: Agrupar por 'CIDADE' e somar 'VALOR_FINAL'
  df_final = df_concatenado.groupby('CIDADE', as_index=False)['VALOR_FINAL'].sum()
  return df_final

## Acesso aos indicadores de criticidade

In [266]:
xlxs_criticidade = pd.ExcelFile('/content/drive/MyDrive/BCC/BRUNA_LIMA/DATASETS/IVSA/IND_CRITICIDADE.xlsx')

In [267]:
dfs_criticidade = normalizar_dataframe(xlxs_criticidade)

In [268]:
df_test = dfs_criticidade['Ind1']
df_test.head(10)

,CIDADE,VALOR,Z-SCORE,OUTLIER,VALOR_NORMAL
0,Angra dos Reis,8297,-0.043196,False,0.154303
1,Aperibé,541,-0.239053,False,0.005172
2,Araruama,6583,-0.086479,False,0.121347
3,Areal,606,-0.237412,False,0.006422
4,Armação dos Búzios,1621,-0.211781,False,0.025938
5,Arraial do Cabo,1517,-0.214407,False,0.023939
6,Barra do Piraí,5815,-0.105873,False,0.106580
7,Barra Mansa,9827,-0.004560,False,0.183722
8,Belford Roxo,27469,0.440941,False,0.522939
9,Bom Jardim,1312,-0.219584,False,0.019997


Aplicar os pesos AHP

In [269]:
# Itera sobre os DataFrames e aplica o peso correspondente
for sheet_name, df in dfs_criticidade.items():
  if sheet_name in criticidade.target_weights:  # Verifica se a aba tem um peso definido
    peso = criticidade.target_weights[sheet_name] # Pega o peso correspondente
    df['VALOR_FINAL'] = df['VALOR_NORMAL'] * peso  # Aplica o peso

In [270]:
df_test = dfs_criticidade['Ind1']
df_test.head(10)

,CIDADE,VALOR,Z-SCORE,OUTLIER,VALOR_NORMAL,VALOR_FINAL
0,Angra dos Reis,8297,-0.043196,False,0.154303,0.019597
1,Aperibé,541,-0.239053,False,0.005172,0.000657
2,Araruama,6583,-0.086479,False,0.121347,0.015411
3,Areal,606,-0.237412,False,0.006422,0.000816
4,Armação dos Búzios,1621,-0.211781,False,0.025938,0.003294
5,Arraial do Cabo,1517,-0.214407,False,0.023939,0.003040
6,Barra do Piraí,5815,-0.105873,False,0.106580,0.013536
7,Barra Mansa,9827,-0.004560,False,0.183722,0.023333
8,Belford Roxo,27469,0.440941,False,0.522939,0.066413
9,Bom Jardim,1312,-0.219584,False,0.019997,0.002540


Valor total de criticidade

In [271]:
df_global_criticidade = sum_criterio(dfs_criticidade)

In [272]:
df_global_criticidade.head(10)

,CIDADE,VALOR_FINAL
0,Angra dos Reis,0.380436
1,Aperibé,0.026068
2,Araruama,0.200541
3,Areal,0.056892
4,Armação dos Búzios,0.053204
5,Arraial do Cabo,0.082699
6,Barra Mansa,0.165062
7,Barra do Piraí,0.101631
8,Belford Roxo,0.510232
9,Bom Jardim,0.045854


## Acesso aos indicadores de suportabilidade

In [273]:
xlxs_suportabilidade = pd.ExcelFile('/content/drive/MyDrive/BCC/BRUNA_LIMA/DATASETS/IVSA/IND_SUPORTABILIDADE.xlsx')

In [274]:
dfs_suportabilidade = normalizar_dataframe(xlxs_suportabilidade)

In [275]:
df_test = dfs_suportabilidade['Ind9']
df_test.head(10)

,CIDADE,VALOR,Z-SCORE,OUTLIER,VALOR_NORMAL
0,Angra dos Reis,406,-0.252266,False,0.042037
1,Aperibé,168,-0.330302,False,0.016579
2,Araruama,1522,0.113655,False,0.161408
3,Areal,59,-0.366042,False,0.004920
4,Armação dos Búzios,48,-0.369649,False,0.003744
5,Arraial do Cabo,46,-0.370305,False,0.003530
6,Barra do Piraí,631,-0.178491,False,0.066103
7,Barra Mansa,171,-0.329319,False,0.016900
9,Bom Jardim,173,-0.328663,False,0.017114
10,Bom Jesus do Itabapoana,601,-0.188328,False,0.062894


Aplicar os pesos AHP

In [276]:
# Itera sobre os DataFrames e aplica o peso correspondente
for sheet_name, df in dfs_suportabilidade.items():
  if sheet_name in suporte.target_weights:  # Verifica se a aba tem um peso definido
    peso = suporte.target_weights[sheet_name] # Pega o peso correspondente
    df['VALOR_FINAL'] = df['VALOR_NORMAL'] * peso  # Aplica o peso

In [277]:
df_test = dfs_suportabilidade['Ind9']
df_test.head(10)

,CIDADE,VALOR,Z-SCORE,OUTLIER,VALOR_NORMAL,VALOR_FINAL
0,Angra dos Reis,406,-0.252266,False,0.042037,0.013199
1,Aperibé,168,-0.330302,False,0.016579,0.005206
2,Araruama,1522,0.113655,False,0.161408,0.050682
3,Areal,59,-0.366042,False,0.004920,0.001545
4,Armação dos Búzios,48,-0.369649,False,0.003744,0.001176
5,Arraial do Cabo,46,-0.370305,False,0.003530,0.001108
6,Barra do Piraí,631,-0.178491,False,0.066103,0.020756
7,Barra Mansa,171,-0.329319,False,0.016900,0.005307
9,Bom Jardim,173,-0.328663,False,0.017114,0.005374
10,Bom Jesus do Itabapoana,601,-0.188328,False,0.062894,0.019749


Valor total suportabilidade

In [278]:
df_global_suportabilidade = sum_criterio(dfs_suportabilidade)

In [279]:
df_global_suportabilidade.head(10)

,CIDADE,VALOR_FINAL
0,Angra dos Reis,0.146767
1,Aperibé,0.057269
2,Araruama,0.270937
3,Areal,0.037520
4,Armação dos Búzios,0.113074
5,Arraial do Cabo,0.119333
6,Barra Mansa,0.104106
7,Barra do Piraí,0.153934
8,Belford Roxo,0.216415
9,Bom Jardim,0.088888


## Acesso aos indicadores do Histórico de Incidentes

In [280]:
xlxs_historico = pd.ExcelFile('/content/drive/MyDrive/BCC/BRUNA_LIMA/DATASETS/IVSA/IND_HISTORICO.xlsx')

In [281]:
dfs_historico = normalizar_dataframe(xlxs_historico)

In [282]:
df_test = dfs_historico['Ind14']
df_test.head(10)

,CIDADE,VALOR,Z-SCORE,OUTLIER,VALOR_NORMAL
0,Angra dos Reis,19,1.330881,False,0.782609
1,Aperibé,7,-0.194398,False,0.260870
2,Araruama,2,-0.829931,False,0.043478
3,Areal,2,-0.829931,False,0.043478
4,Armação dos Búzios,1,-0.957038,False,0.000000
5,Arraial do Cabo,14,0.695348,False,0.565217
6,Barra do Piraí,13,0.568241,False,0.521739
7,Barra Mansa,7,-0.194398,False,0.260870
8,Belford Roxo,4,-0.575718,False,0.130435
9,Bom Jesus do Itabapoana,20,1.457987,False,0.826087


Aplicação dos pesos AHP

In [283]:
# Itera sobre os DataFrames e aplica o peso correspondente
for sheet_name, df in dfs_historico.items():
  if sheet_name in historico.target_weights:  # Verifica se a aba tem um peso definido
    peso = historico.target_weights[sheet_name] # Pega o peso correspondente
    df['VALOR_FINAL'] = df['VALOR_NORMAL'] * peso  # Aplica o peso

In [284]:
df_test = dfs_historico['Ind14']
df_test.head(10)

,CIDADE,VALOR,Z-SCORE,OUTLIER,VALOR_NORMAL,VALOR_FINAL
0,Angra dos Reis,19,1.330881,False,0.782609,0.102522
1,Aperibé,7,-0.194398,False,0.260870,0.034174
2,Araruama,2,-0.829931,False,0.043478,0.005696
3,Areal,2,-0.829931,False,0.043478,0.005696
4,Armação dos Búzios,1,-0.957038,False,0.000000,0.000000
5,Arraial do Cabo,14,0.695348,False,0.565217,0.074043
6,Barra do Piraí,13,0.568241,False,0.521739,0.068348
7,Barra Mansa,7,-0.194398,False,0.260870,0.034174
8,Belford Roxo,4,-0.575718,False,0.130435,0.017087
9,Bom Jesus do Itabapoana,20,1.457987,False,0.826087,0.108217


Soma total Histórico de Incidentes

In [285]:
df_global_historico = sum_criterio(dfs_historico)

In [286]:
df_global_historico.head(10)

,CIDADE,VALOR_FINAL
0,Angra dos Reis,0.431239
1,Aperibé,0.073242
2,Araruama,0.031031
3,Areal,0.010479
4,Armação dos Búzios,0.013379
5,Arraial do Cabo,0.080295
6,Barra Mansa,0.118923
7,Barra do Piraí,0.191746
8,Belford Roxo,0.475718
9,Bom Jesus do Itabapoana,0.420153


# Aplicação dos pesos AHP aos indicadores - Sem município Rio de Janeiro

A ideia é investigar como ficará o resultado removendo o maior município em relação a quantidade de pessoas (6.729.894)

## Auxiliares

In [296]:
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [297]:
minmax_scaler = MinMaxScaler(feature_range=(0, 1))
robust_scaler = RobustScaler()

In [298]:
df_global = {}

In [299]:
def normalizar_dataframe(xlsx):
    df_global = {}

    # Itera sobre todas as abas do arquivo Excel
    for sheet_name in xlsx.sheet_names:
      # Lê cada aba e armazena no dicionário
      df = xlsx.parse(sheet_name)

      # Verifica se a coluna "CIDADE" existe no DataFrame
      if 'CIDADE' in df.columns:
          # Remove o " (RJ)" da coluna "CIDADE"
          df['CIDADE'] = df['CIDADE'].str.replace(' (RJ)', '', regex=False)

      if 'VALOR' in df.columns:
        # 1) Aplicar RobustScaler que usa a mediana e o intervalo interquartil (IQR), sendo resistente a outliers sem distorcer os dados.
        # link: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.RobustScaler.html
        df["VALOR_ROBUST"] = robust_scaler.fit_transform(df[["VALOR"]])

        # 2) Normalização Min-Max (após escalonamento robusto)
        # link: https://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.MinMaxScaler.html
        df['VALOR_NORMAL'] = minmax_scaler.fit_transform(df[['VALOR_ROBUST']])

      df_sem_rj = df[df['CIDADE'] != 'Rio de Janeiro'].copy()

      # Armazena o DataFrame no dicionário
      df_global[sheet_name] = df_sem_rj

    return df_global

In [300]:
def sum_criterio(dfs_dic):
  # Passo 1: Concatenar todos os DataFrames em um único DataFrame
  df_concatenado = pd.concat(dfs_dic.values(), ignore_index=True)
  # Passo 2: Agrupar por 'CIDADE' e somar 'VALOR_FINAL'
  df_final = df_concatenado.groupby('CIDADE', as_index=False)['VALOR_FINAL'].sum()
  return df_final

## Acesso ao indicadores de criticidade

In [301]:
xlxs_criticidade = pd.ExcelFile('/content/drive/MyDrive/BCC/BRUNA_LIMA/DATASETS/IVSA/IND_CRITICIDADE.xlsx')

In [302]:
dfs_criticidade = normalizar_dataframe(xlxs_criticidade)

In [303]:
df_test = dfs_criticidade['Ind1']
df_test.head(10)

,CIDADE,VALOR,VALOR_ROBUST,VALOR_NORMAL
0,Angra dos Reis,8297,1.085693,0.021375
1,Aperibé,541,-0.213687,0.000716
2,Araruama,6583,0.798542,0.016809
3,Areal,606,-0.202798,0.000890
4,Armação dos Búzios,1621,-0.032753,0.003593
5,Arraial do Cabo,1517,-0.050176,0.003316
6,Barra do Piraí,5815,0.669878,0.014764
7,Barra Mansa,9827,1.342017,0.025450
8,Belford Roxo,27469,4.297621,0.072439
9,Bom Jardim,1312,-0.084520,0.002770


Aplicar os pesos do AHP

In [304]:
# Itera sobre os DataFrames e aplica o peso correspondente
for sheet_name, df in dfs_criticidade.items():
  if sheet_name in criticidade.target_weights:  # Verifica se a aba tem um peso definido
    peso = criticidade.target_weights[sheet_name] # Pega o peso correspondente
    df['VALOR_FINAL'] = df['VALOR_NORMAL'] * peso  # Aplica o peso

In [305]:
df_test = dfs_criticidade['Ind1']
df_test.head(10)

,CIDADE,VALOR,VALOR_ROBUST,VALOR_NORMAL,VALOR_FINAL
0,Angra dos Reis,8297,1.085693,0.021375,0.002715
1,Aperibé,541,-0.213687,0.000716,0.000091
2,Araruama,6583,0.798542,0.016809,0.002135
3,Areal,606,-0.202798,0.000890,0.000113
4,Armação dos Búzios,1621,-0.032753,0.003593,0.000456
5,Arraial do Cabo,1517,-0.050176,0.003316,0.000421
6,Barra do Piraí,5815,0.669878,0.014764,0.001875
7,Barra Mansa,9827,1.342017,0.025450,0.003232
8,Belford Roxo,27469,4.297621,0.072439,0.009200
9,Bom Jardim,1312,-0.084520,0.002770,0.000352


Valor total do critério criticidade

In [306]:
df_global_criticidade = sum_criterio(dfs_criticidade)

In [307]:
df_global_criticidade.head(10)

,CIDADE,VALOR_FINAL
0,Angra dos Reis,0.066839
1,Aperibé,0.023525
2,Araruama,0.053583
3,Areal,0.052738
4,Armação dos Búzios,0.031745
5,Arraial do Cabo,0.035618
6,Barra Mansa,0.050505
7,Barra do Piraí,0.038431
8,Belford Roxo,0.089259
9,Bom Jardim,0.035152


## Acesso ao indicadores de suportabilidade

In [308]:
xlxs_suportabilidade = pd.ExcelFile('/content/drive/MyDrive/BCC/BRUNA_LIMA/DATASETS/IVSA/IND_SUPORTABILIDADE.xlsx')

In [309]:
dfs_suportabilidade = normalizar_dataframe(xlxs_suportabilidade)

In [310]:
df_test = dfs_suportabilidade['Ind9']
df_test.head(10)

,CIDADE,VALOR,VALOR_ROBUST,VALOR_NORMAL
0,Angra dos Reis,406,0.096705,0.019418
1,Aperibé,168,-0.310655,0.007658
2,Araruama,1522,2.006846,0.074559
3,Areal,59,-0.497219,0.002273
4,Armação dos Búzios,48,-0.516046,0.001729
5,Arraial do Cabo,46,-0.519469,0.001631
6,Barra do Piraí,631,0.481814,0.030535
7,Barra Mansa,171,-0.305520,0.007807
8,Belford Roxo,20252,34.065041,1.000000
9,Bom Jardim,173,-0.302097,0.007906


In [311]:
# Itera sobre os DataFrames e aplica o peso correspondente
for sheet_name, df in dfs_suportabilidade.items():
  if sheet_name in suporte.target_weights:  # Verifica se a aba tem um peso definido
    peso = suporte.target_weights[sheet_name] # Pega o peso correspondente
    df['VALOR_FINAL'] = df['VALOR_NORMAL'] * peso  # Aplica o peso

In [312]:
df_test = dfs_suportabilidade['Ind9']
df_test.head(10)

,CIDADE,VALOR,VALOR_ROBUST,VALOR_NORMAL,VALOR_FINAL
0,Angra dos Reis,406,0.096705,0.019418,0.006097
1,Aperibé,168,-0.310655,0.007658,0.002405
2,Araruama,1522,2.006846,0.074559,0.023412
3,Areal,59,-0.497219,0.002273,0.000714
4,Armação dos Búzios,48,-0.516046,0.001729,0.000543
5,Arraial do Cabo,46,-0.519469,0.001631,0.000512
6,Barra do Piraí,631,0.481814,0.030535,0.009588
7,Barra Mansa,171,-0.305520,0.007807,0.002451
8,Belford Roxo,20252,34.065041,1.000000,0.314000
9,Bom Jardim,173,-0.302097,0.007906,0.002482


Valor total do critério suportabilidade

In [313]:
df_global_suportabilidade = sum_criterio(dfs_suportabilidade)

In [314]:
df_global_suportabilidade.head(10)

,CIDADE,VALOR_FINAL
0,Angra dos Reis,0.070031
1,Aperibé,0.036685
2,Araruama,0.129721
3,Areal,0.021860
4,Armação dos Búzios,0.063616
5,Arraial do Cabo,0.075480
6,Barra Mansa,0.052309
7,Barra do Piraí,0.087768
8,Belford Roxo,0.581780
9,Bom Jardim,0.050191


## Acesso ao indicadores de Histórico de Incidente

In [315]:
xlxs_historico = pd.ExcelFile('/content/drive/MyDrive/BCC/BRUNA_LIMA/DATASETS/IVSA/IND_HISTORICO.xlsx')

In [316]:
dfs_historico = normalizar_dataframe(xlxs_historico)

In [317]:
df_test = dfs_historico['Ind14']
df_test.head(10)

,CIDADE,VALOR,VALOR_ROBUST,VALOR_NORMAL
0,Angra dos Reis,19,1.444444,0.375000
1,Aperibé,7,0.111111,0.125000
2,Araruama,2,-0.444444,0.020833
3,Areal,2,-0.444444,0.020833
4,Armação dos Búzios,1,-0.555556,0.000000
5,Arraial do Cabo,14,0.888889,0.270833
6,Barra do Piraí,13,0.777778,0.250000
7,Barra Mansa,7,0.111111,0.125000
8,Belford Roxo,4,-0.222222,0.062500
9,Bom Jesus do Itabapoana,20,1.555556,0.395833


In [318]:
# Itera sobre os DataFrames e aplica o peso correspondente
for sheet_name, df in dfs_historico.items():
  if sheet_name in historico.target_weights:  # Verifica se a aba tem um peso definido
    peso = historico.target_weights[sheet_name] # Pega o peso correspondente
    df['VALOR_FINAL'] = df['VALOR_NORMAL'] * peso  # Aplica o peso

In [319]:
df_test = dfs_historico['Ind14']
df_test.head(10)

,CIDADE,VALOR,VALOR_ROBUST,VALOR_NORMAL,VALOR_FINAL
0,Angra dos Reis,19,1.444444,0.375000,0.049125
1,Aperibé,7,0.111111,0.125000,0.016375
2,Araruama,2,-0.444444,0.020833,0.002729
3,Areal,2,-0.444444,0.020833,0.002729
4,Armação dos Búzios,1,-0.555556,0.000000,0.000000
5,Arraial do Cabo,14,0.888889,0.270833,0.035479
6,Barra do Piraí,13,0.777778,0.250000,0.032750
7,Barra Mansa,7,0.111111,0.125000,0.016375
8,Belford Roxo,4,-0.222222,0.062500,0.008188
9,Bom Jesus do Itabapoana,20,1.555556,0.395833,0.051854


Valor total do critério Histórico de Incidentes

In [320]:
df_global_historico = sum_criterio(dfs_historico)

In [321]:
df_global_historico.head(10)

,CIDADE,VALOR_FINAL
0,Angra dos Reis,0.170567
1,Aperibé,0.028677
2,Araruama,0.014302
3,Areal,0.004374
4,Armação dos Búzios,0.004659
5,Arraial do Cabo,0.038441
6,Barra Mansa,0.048901
7,Barra do Piraí,0.065743
8,Belford Roxo,0.241738
9,Bom Jesus do Itabapoana,0.141883


# Resultados

In [322]:
total = pd.DataFrame(columns=["CIDADE", "VALOR_FINAL"])

In [323]:
# Juntamos criticidade e historico
df_total_aux = pd.merge(
    df_global_criticidade.rename(columns={'VALOR_FINAL': 'valor_criticidade'}),
    df_global_historico.rename(columns={'VALOR_FINAL': 'valor_historico'}),
    on='CIDADE',
    how='outer'
)

# Juntamos suportabilidade
df_total_aux = pd.merge(
    df_total_aux,
    df_global_suportabilidade.rename(columns={'VALOR_FINAL': 'valor_suportabilidade'}),
    on='CIDADE',
    how='outer'
)

# Opcional: caso valores ausentes devam ser tratados como zero
df_total_aux[['valor_criticidade', 'valor_historico', 'valor_suportabilidade']] = df_total_aux[
    ['valor_criticidade', 'valor_historico', 'valor_suportabilidade']
].fillna(0)

# Cálculo: total.valor = criticidade + historico  + suportabilidade
df_total_aux['VALOR_FINAL'] = (
    df_total_aux['valor_criticidade'] + df_total_aux['valor_historico'] + df_total_aux['valor_suportabilidade']
)

# Arredondando a coluna 'VALOR_FINAL' para 2 casas decimais
df_total_aux['VALOR_FINAL'] = df_total_aux['VALOR_FINAL'].round(2)

# Se desejar manter apenas municipio e valor final
df_total = df_total_aux[['CIDADE', 'VALOR_FINAL']]

In [324]:
df_total.head(10)

,CIDADE,VALOR_FINAL
0,Angra dos Reis,0.31
1,Aperibé,0.09
2,Araruama,0.20
3,Areal,0.08
4,Armação dos Búzios,0.10
5,Arraial do Cabo,0.15
6,Barra Mansa,0.15
7,Barra do Piraí,0.19
8,Belford Roxo,0.91
9,Bom Jardim,0.09


In [325]:
df_total.loc[:, 'FAIXA'] = pd.cut(
    df_total['VALOR_FINAL'],
    bins=[0.0, 0.2, 0.4, 0.6, 0.8, 1.0],
    labels=['MBV', 'BV', 'MV', 'AV', 'MAV'],
    right=False
)

df_total = df_total.fillna('MAV')

<ipython-input-325-09bc20ad031e>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_total.loc[:, 'FAIXA'] = pd.cut(


In [326]:
df_total.head(100)

,CIDADE,VALOR_FINAL,FAIXA
0,Angra dos Reis,0.31,BV
1,Aperibé,0.09,MBV
2,Araruama,0.20,BV
3,Areal,0.08,MBV
4,Armação dos Búzios,0.10,MBV
...,...,...,...
86,Três Rios,0.08,MBV
87,Valença,0.10,MBV
88,Varre-Sai,0.14,MBV
89,Vassouras,0.09,MBV


Baixar resultados

In [328]:
# Criação de uma lista com os DataFrames
dfs_finais = {
    "criticidade": df_global_criticidade,
    "suportabilidade": df_global_suportabilidade,
    "historico": df_global_historico,
    "total": df_total
}

In [329]:
from google.colab import files
import xlsxwriter # Import the module

with pd.ExcelWriter('RESULTADO_SEM_RJ.xlsx', engine='xlsxwriter') as writer:
  for sheet_name, df in dfs_finais.items():
    df.to_excel(writer, sheet_name=sheet_name, index=False)

# Para baixar o arquivo no Colab:
files.download('RESULTADO_SEM_RJ.xlsx')

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>